In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'   # hides INFO/WARNING logs from TF C++
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, losses
import numpy as np

In [2]:
train_dataset = image_dataset_from_directory(
    r"/home/hamid/ML/Datasets/Men-Women-small/train",
    image_size = (224 , 224),
    batch_size = 32 ,
    label_mode = "binary"
)

validation_dataset = image_dataset_from_directory(
    r"/home/hamid/ML/Datasets/Men-Women-small/validation",
    image_size = (224 , 224),
    batch_size = 32 ,
    label_mode = "binary"
)

test_dataset = image_dataset_from_directory(
    r"/home/hamid/ML/Datasets/Men-Women-small/test",
    image_size = (224 , 224),
    batch_size = 32 ,
    label_mode = "binary"
)

Found 2000 files belonging to 2 classes.


I0000 00:00:1765043824.447040  274762 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5558 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:01:00.0, compute capability: 8.9


Found 400 files belonging to 2 classes.
Found 400 files belonging to 2 classes.


In [3]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

In [4]:
# Build VGGNet model with Functional API
inputs = keras.Input(shape=(224, 224, 3))

# Data augmentation and preprocessing
x = data_augmentation(inputs)
x = layers.Rescaling(1.0 / 255)(x)

# Block 1
x = layers.Conv2D(64, 3, strides=1, padding='same', activation='relu')(x)
x = layers.Conv2D(64, 3, strides=1, padding='same', activation='relu')(x)
x = layers.MaxPooling2D(2, strides=2)(x)

# Block 2
x = layers.Conv2D(128, 3, strides=1, padding='same', activation='relu')(x)
x = layers.Conv2D(128, 3, strides=1, padding='same', activation='relu')(x)
x = layers.MaxPooling2D(2, strides=2)(x)

# Block 3
x = layers.Conv2D(256, 3, strides=1, padding='same', activation='relu')(x)
x = layers.Conv2D(256, 3, strides=1, padding='same', activation='relu')(x)
x = layers.Conv2D(256, 3, strides=1, padding='same', activation='relu')(x)
x = layers.MaxPooling2D(2, strides=2)(x)

# Block 4
x = layers.Conv2D(512, 3, strides=1, padding='same', activation='relu')(x)
x = layers.Conv2D(512, 3, strides=1, padding='same', activation='relu')(x)
x = layers.Conv2D(512, 3, strides=1, padding='same', activation='relu')(x)
x = layers.MaxPooling2D(2, strides=2)(x)

# Block 5
x = layers.Conv2D(512, 3, strides=1, padding='same', activation='relu')(x)
x = layers.Conv2D(512, 3, strides=1, padding='same', activation='relu')(x)
x = layers.Conv2D(512, 3, strides=1, padding='same', activation='relu')(x)
x = layers.MaxPooling2D(2, strides=2)(x)

# Fully connected layers
x = layers.Flatten()(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# Output layer (adjust based on your task)
# For 10-class classification:
outputs = layers.Dense(1, activation="sigmoid")(x)

# For binary classification, use:
# outputs = layers.Dense(1, activation='sigmoid')(x)

# Create model
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,31

 Total params: 134,264,641 (512.18 MB)

 Trainable params: 134,264,641 (512.18 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    metrics=["accuracy"],
)

# Train with appropriate batch size
history = model.fit(
    train_dataset,
    batch_size=32,
    epochs=40,
    validation_data=validation_dataset
)

Epoch 1/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 39s 462ms/step - accuracy: 0.4900 - loss: 0.6938 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 26s 409ms/step - accuracy: 0.4815 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 3/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 26s 408ms/step - accuracy: 0.4835 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/40
47/63 ━━━━━━━━━━━━━━━━━━━━ 6s 386ms/step - accuracy: 0.4900 - loss: 0.6932

In [ ]:
# Evaluate
test_loss, test_acc = model.evaluate(test_dataset)
print(f'\nTest accuracy: {test_acc:.4f}')